In [19]:
import random
import numpy as np
import gurobipy as gp

# Room draw optimization models using gurobi python API
## 1. Model for multiple preferences with individual parties
Consistent with the output in multi_preferences model in `multi_preferences.mod`

In [105]:
# number parameters
n_rooms = 20
n_people = 20
n_preferences = 5

ROOMS = list(range(1, n_rooms + 1))
PEOPLE = list(range(1, n_people + 1))
PREFERENCES = list(range(1, n_preferences + 1))

# preference and weight parameters
weights = {(p, pr): n_preferences - pr  + 1 for p in PEOPLE for pr in PREFERENCES}



# preferences = np.random.randint(low=1,high=n_rooms,size=(n_people, n_preferences))
preferences = np.array([
       [ 5,  7, 19, 13,  9],
       [16,  6,  3,  6, 18],
       [14,  2, 15,  7, 14],
       [15,  2,  4,  2, 16],
       [18,  5, 18, 13,  5],
       [ 3,  1,  5, 15,  3],
       [ 2,  5,  8,  6,  9],
       [13,  9, 12,  1,  5],
       [19, 17,  6, 13, 10],
       [ 5, 17, 19,  5, 14],
       [ 5, 17, 12,  1, 15],
       [16, 18,  4,  4,  4],
       [ 9,  8, 13,  4, 15],
       [ 6,  8,  3, 11, 14],
       [12, 18, 19, 16, 14],
       [17,  4, 12,  6, 13],
       [16, 17, 13,  3, 14],
       [ 3, 12, 17,  4,  1],
       [ 4, 12,  3, 11, 11],
       [14, 14, 16, 12, 18]])


In [106]:
m = gp.Model("Multi-Preference Model")
satisfied = m.addVars(PEOPLE, PREFERENCES, vtype=gp.GRB.BINARY)
assignment = m.addVars(PEOPLE, ROOMS, vtype=gp.GRB.BINARY)
room_number = m.addVars(PEOPLE, lb=1, vtype=gp.GRB.INTEGER)

m.addConstrs((satisfied[p,pr] == assignment[p,preferences[p-1,pr-1]] for p in PEOPLE for pr in PREFERENCES),name='Preference Satisfied')
m.addConstrs((assignment.sum('*', r) == 1 for r in ROOMS), name='No overlaps')

rooms_dict = {(p, r): r for p in PEOPLE for r in ROOMS }
m.addConstrs((assignment.prod(rooms_dict, p, '*') == room_number[p] for p in PEOPLE)
, name='Assignment number')

m.setObjective(satisfied.prod(weights, '*', '*'), gp.GRB.MAXIMIZE)


In [107]:
m.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 140 rows, 520 columns and 1020 nonzeros
Model fingerprint: 0x4c7a5c8a
Variable types: 0 continuous, 520 integer (500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 100 rows and 120 columns
Presolve time: 0.00s
Presolved: 40 rows, 400 columns, 800 nonzeros
Variable types: 0 continuous, 400 integer (400 binary)
Found heuristic solution: objective -0.0000000

Root relaxation: objective 9.600000e+01, 31 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0        

## 2. Model for range preferences
In this model, we try to implement preferences for not a particular room but a general 

## 3. Model for macro groups
In this model, consider preferences for dorm suites

## 4. Model for realistic draw order
In thin model, consider the ordering and preference of selection